In [2]:
!pip install scikit-hts

Defaulting to user installation because normal site-packages is not writeable


### try scikit_hts with a naive hts (1 observation for n hierarchies)

In [2]:
import numpy as np
import pandas as pd 
import hts  # To install: pip install scikit-hts
import collections
from scipy.optimize import lsq_linear

In [3]:
# a naive hts
hts_df = pd.DataFrame([{'total': 2, 
                        'CA': 1.4, 'TX': 1.8, 'WI': 1.9, 
                        'CA_1': 0.8, 'CA_2': 0.6, 'CA_3': 0.9, 'CA_4': 0.3,
                        'TX_1': 0.03, 'TX_2': 0.5, 'TX_3': 0.5, 
                        'WI_1': 1.6, 'WI_2': 1.2, 'WI_3': 1.5
                        }])

In [4]:
hts_df

,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,2,1.4,1.8,1.9,0.8,0.6,0.9,0.3,0.03,0.5,0.5,1.6,1.2,1.5


In [5]:
states = ['CA', 'TX', 'WI']
stores = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

# build the hierarchy tree as a dictionary

total = {'total': list(states)}
state_h = {k: [v for v in stores if v.startswith(k)] for k in states}
hierarchy = {**total, **state_h}

In [6]:
hierarchy

{'total': ['CA', 'TX', 'WI'],
 'CA': ['CA_1', 'CA_2', 'CA_3', 'CA_4'],
 'TX': ['TX_1', 'TX_2', 'TX_3'],
 'WI': ['WI_1', 'WI_2', 'WI_3']}

In [7]:
tree = hts.hierarchy.HierarchyTree.from_nodes(nodes=hierarchy, df=hts_df)
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(tree)

In [8]:
tree

- total
   |- CA
   |  |- CA_1
   |  |- CA_2
   |  |- CA_3
   |  - CA_4
   |- TX
   |  |- TX_1
   |  |- TX_2
   |  - TX_3
   - WI
      |- WI_1
      |- WI_2
      - WI_3

In [9]:
pred_dict = collections.OrderedDict()
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=hts_df[label].values, columns=['yhat'])

In [10]:
# perform forecast reconciliation
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='OLS', mse={})
revised_forecasts = pd.DataFrame(data=revised,
                                 index=hts_df.index,
                                 columns=sum_mat_labels)
print(revised_forecasts)

      total        WI        TX        CA      CA_1      CA_2      CA_3  \
0  3.135606  1.648295  0.755795  0.731515  0.332879  0.132879  0.432879   

       CA_4      TX_1      TX_2      TX_3      WI_1      WI_2      WI_3  
0 -0.167121 -0.061402  0.408598  0.408598  0.716098  0.316098  0.616098  


In [11]:
#  For non-negative reconciled forecasts
hat_mat = hts.functions.y_hat_matrix(pred_dict)
revised_nnls = np.dot(sum_mat, lsq_linear(sum_mat, hat_mat.flatten(), bounds=(0, np.inf))['x'])
revised_forecasts_nnls = pd.DataFrame(data=revised_nnls.reshape(1,-1),
                                 index=hts_df.index,
                                 columns=sum_mat_labels)

In [12]:
print(revised_forecasts_nnls)

      total        WI        TX        CA      CA_1      CA_2      CA_3  \
0  3.155263  1.633553  0.763158  0.758553  0.286184  0.086184  0.386184   

           CA_4          TX_1      TX_2      TX_3      WI_1      WI_2  \
0  9.425493e-22  5.896338e-17  0.381579  0.381579  0.711184  0.311184   

       WI_3  
0  0.611184  


### try hts with 28 observations at different hierarchies

In [13]:
# load m5 dataset
m5_input_path = "/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy"
sell_price = pd.read_csv(f'{m5_input_path}/sell_prices.csv')
calendar = pd.read_csv(f'{m5_input_path}/calendar.csv')
train = pd.read_csv(f'{m5_input_path}/sales_train_evaluation.csv').set_index('id')
sample_sub = pd.read_csv(f'{m5_input_path}/sample_submission.csv')

In [14]:
cat_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
ts_cols = [col for col in train.columns if col not in cat_cols]
ts_dict = {t: int(t[2:]) for t in ts_cols}

all_sales = pd.DataFrame(train[ts_cols].sum()).transpose()
all_sales['id_str'] = 'all'
all_sales = all_sales[ ['id_str'] +  [c for c in all_sales if c not in ['id_str']] ]

state_sales = train.groupby('state_id',as_index=False)[ts_cols].sum()
state_sales['id_str'] = state_sales['state_id'] 
state_sales = state_sales[ ['id_str'] +  [c for c in state_sales if c not in ['id_str']] ]
state_sales = state_sales.drop(['state_id'],axis=1)

store_sales = train.groupby('store_id',as_index=False)[ts_cols].sum()
store_sales['id_str'] = store_sales['store_id'] 
store_sales = store_sales[ ['id_str'] +  [c for c in store_sales if c not in ['id_str']] ]
store_sales = store_sales.drop(['store_id'],axis=1)

In [15]:
aggregates = pd.concat([all_sales,state_sales,store_sales],ignore_index=True)

In [16]:
true_last_28 = aggregates.set_index('id_str').transpose().rename(columns={'all':'total'}).iloc[-28:]

In [17]:
true_last_28.head()

id_str,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,38793,17524,10662,10607,4472,3926,6359,2767,3076,3883,3703,3166,4178,3263
d_1915,35487,15012,9933,10542,3703,3525,5289,2495,2853,3502,3578,3194,4148,3200
d_1916,34445,14836,9575,10034,3715,3527,5065,2529,2984,3256,3335,3267,3805,2962
d_1917,34732,14664,9655,10413,3618,3754,5015,2277,2664,3441,3550,3201,4342,2870
d_1918,42896,17180,12162,13554,4573,4382,5705,2520,3687,4023,4452,4143,5719,3692


In [18]:
# add noise to true sales numbers to get synthetic prediction data
mu, sigma = 1000, 2000
noise = np.random.normal(mu, sigma, [true_last_28.shape[0], true_last_28.shape[1]]) 
synthetic_pred_28 = true_last_28 + noise

In [19]:
synthetic_pred_28.head()

id_str,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,40794.107116,20310.249689,9283.261608,11004.055540,5487.305805,5498.787862,3046.911172,5198.534020,3620.383902,6877.755948,45.797552,3516.799089,5779.673448,3984.358339
d_1915,34954.648950,16808.107388,12793.262251,16312.313205,8743.859649,3365.877645,3628.556072,-386.036195,4728.552996,5017.702522,4561.018637,4068.695907,7234.410460,8756.880836
d_1916,31132.874621,14140.749744,5109.081951,11520.903229,3955.897598,4568.081110,4831.232509,-617.420420,5956.799642,5879.099755,7212.128420,1366.439263,2462.405932,5427.143101
d_1917,30368.712971,12122.124895,13258.973825,14447.428672,3684.577498,7045.110391,6479.014577,2035.247833,3591.433434,3712.474341,5918.338548,5166.654426,4724.959570,5255.415623
d_1918,40114.806980,14185.994233,14510.862423,15011.000810,4615.695121,3759.690174,6805.132582,1901.906411,7628.832797,7444.392874,4463.836759,9149.140924,6377.094184,7713.072764


In [20]:
tree = hts.hierarchy.HierarchyTree.from_nodes(nodes=hierarchy, df=synthetic_pred_28)
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(tree)

In [21]:
pred_dict = collections.OrderedDict()
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=synthetic_pred_28[label].values, columns=['yhat'])

In [22]:
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='OLS', mse={})
revised_forecasts = pd.DataFrame(data=revised,
                                 index=synthetic_pred_28.index,
                                 columns=sum_mat_labels)

In [23]:
revised_forecasts
# we could see there are negative forecast values, e.g. CA_4 at day_1915, 1916, 1917

,total,WI,TX,CA,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,40937.161764,11465.958389,9491.139571,19980.063805,5674.437041,5685.919098,3234.042409,5385.665256,3269.451292,6526.823338,-305.135058,2911.841593,5174.715952,3379.400843
d_1915,38585.947533,14525.757767,10448.291289,13611.898477,8308.769975,2930.787972,3193.466399,-821.125869,3442.225374,3731.374900,3274.691015,2223.952761,5389.667315,6912.137691
d_1916,31822.503786,10437.452621,8076.596543,13308.454622,4098.563554,4710.747066,4973.898465,-474.754464,2299.655884,2221.955997,3554.984662,1760.260705,2856.227374,5820.964543
d_1917,33717.166869,12110.988485,10738.451526,10867.726857,1590.521638,4951.054531,4384.958717,-58.808027,2763.501835,2884.542742,5090.406949,4154.640714,3712.945859,4243.401912
d_1918,42383.280697,15366.722288,14066.057138,12950.501271,3582.714367,2726.709420,5772.151828,868.925657,5805.164366,5620.724443,2640.168328,6524.945729,3752.898990,5088.877569
d_1919,51601.373469,13373.280000,11936.010010,26292.083459,4541.715661,7141.155700,9510.166113,5099.045985,3292.926736,3148.823202,5494.260072,4034.954384,3400.342105,5937.983511
d_1920,54544.717948,18086.853508,11610.560099,24847.304341,5621.654733,5442.106717,10428.873871,3354.669020,4733.900979,3406.296469,3470.362651,4376.204460,7941.360317,5769.288731
d_1921,46068.220944,14292.601174,15037.178545,16738.441226,6127.582697,2623.874920,4764.057547,3222.926062,971.339157,5639.495588,8426.343799,3682.824451,5064.559393,5545.217329
d_1922,49848.880603,16837.249175,12478.377644,20533.253784,1526.485015,6300.784533,8493.805431,4212.178806,1466.441737,4060.945311,6950.990597,2852.926930,7819.594871,6164.727374
d_1923,39686.479542,12974.584513,9404.295213,17307.599816,5875.921633,701.847159,8304.898593,2424.932431,366.167421,4873.477092,4164.650700,3636.986096,4720.779912,4616.818505


In [24]:
# add non-negative constraint when reconciliate
hat_mat = hts.functions.y_hat_matrix(pred_dict)

In [25]:
b_mat = []
for i in range(28):
    b_mat.append(lsq_linear(sum_mat, hat_mat[i].flatten(), bounds=(0, np.inf))['x'])

In [26]:
b_mat_final = np.stack(b_mat).transpose()
revised_nnls = np.dot(sum_mat, b_mat_final).transpose()
revised_forecasts_nnls = pd.DataFrame(data=revised_nnls,
                                 index=synthetic_pred_28.index,
                                 columns=sum_mat_labels)

In [27]:
revised_forecasts_nnls
# the previously negative forecasts are now close to 0 (CA_$ at d_1915, d_1916, d_1917)

,total,WI,TX,CA,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,40968.781977,11442.243229,9571.771115,19954.767634,5668.112999,5679.595056,3227.718366,5.379341e+03,3.157200e+03,6414.571581,4.601192e-24,2.903937e+03,5166.810899,3371.495790
d_1915,38649.111062,14478.385121,10400.918643,13769.807298,8087.697626,2709.715623,2972.394049,2.155322e-22,3.426434e+03,3715.584018,3.258900e+03,2.208162e+03,5373.876433,6896.346808
d_1916,31859.023360,10410.062941,8049.206862,13399.753557,3970.745045,4582.928557,4846.079956,4.032505e-23,2.290526e+03,2212.826103,3.545855e+03,1.751131e+03,2847.097480,5811.834650
d_1917,33721.690563,12107.595714,10735.058755,10879.036093,1574.688707,4935.221600,4369.125786,8.430783e-12,2.762371e+03,2883.411818,5.089276e+03,4.153510e+03,3711.814935,4242.270988
d_1918,42383.280697,15366.722288,14066.057138,12950.501271,3582.714367,2726.709420,5772.151828,8.689257e+02,5.805164e+03,5620.724443,2.640168e+03,6.524946e+03,3752.898990,5088.877569
d_1919,51601.373469,13373.280000,11936.010010,26292.083459,4541.715661,7141.155700,9510.166113,5.099046e+03,3.292927e+03,3148.823202,5.494260e+03,4.034954e+03,3400.342105,5937.983511
d_1920,54544.717948,18086.853508,11610.560099,24847.304341,5621.654733,5442.106717,10428.873871,3.354669e+03,4.733901e+03,3406.296469,3.470363e+03,4.376204e+03,7941.360317,5769.288731
d_1921,46068.220944,14292.601174,15037.178545,16738.441226,6127.582697,2623.874920,4764.057547,3.222926e+03,9.713392e+02,5639.495588,8.426344e+03,3.682824e+03,5064.559393,5545.217329
d_1922,49848.880603,16837.249175,12478.377644,20533.253784,1526.485015,6300.784533,8493.805431,4.212179e+03,1.466442e+03,4060.945311,6.950991e+03,2.852927e+03,7819.594871,6164.727374
d_1923,39686.479542,12974.584513,9404.295213,17307.599816,5875.921633,701.847159,8304.898593,2.424932e+03,3.661674e+02,4873.477092,4.164651e+03,3.636986e+03,4720.779912,4616.818505
